In [15]:
import os
proxy_address = "http://172.27.9.200:15672"
os.environ["http_proxy"] = proxy_address
os.environ["HTTP_PROXY"] = proxy_address
os.environ["https_proxy"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address

from opencc import OpenCC
import autogen
from preload_settings import *
from langchain.prompts import PromptTemplate

In [2]:
from autogen.retrieve_utils import (
    extract_text_from_pdf, 
    get_file_from_url
)

- 结论1: SEC的htm文件无法直接用`requests.get`下载，会报`403错误`: "https://www.sec.gov/Archives/edgar/data/1652044/000165204423000070/goog-20230630.htm"
- 结论2: 港股的PDF公告，都可以用`autogen.retrieve_utils.get_file_from_url()`下载成pdf文件, 存放在本地路径

In [10]:
download_dir = "/Users/jarvixwang/Documents/Project/autogen/download"
os.makedirs(download_dir, exist_ok=True)
# "https://news.futunn.com/en/post/32850327",
save_path = get_file_from_url("https://www1.hkexnews.hk/listedco/listconews/sehk/2023/1015/2023101500054_c.pdf",
                               save_path=os.path.join(download_dir, "hk_notice.pdf"))


In [9]:
print(extract_text_from_pdf(save_path))

香港交易及結算所有限公司及香港聯合交易所有限公司對本公告之內容概不負責，對其準確性或完整性亦不
發表任何聲明，並明確表示，概不對因本公告全部或任何部分內容而產生或因倚賴該等內容而引致之任何損
失承擔任何責任。
Smoore International Holdings Limited
思摩爾國際控股有限公司
（於開曼群島註冊成立的有限公司）
（股份代號：6969）
內幕消息
上市前煙草產品申請更新
本公告乃由思摩爾國際控股有限公司（「本公司」，連同其附屬公司統稱「本集團」）董事（「董
事」）會（「董事會」）根據香港聯合交易所有限公司證券上市規則（「上市規則」）第13.09條及香
港法例第571章證券及期貨條例第XIVA部內幕消息條文（定義見上市規則）作出。
正如本公司2022年年度報告及2023年中期報告中所列述，美國聯邦立法要求電子尼古丁傳
送系統產品需要符合美國食品藥品監督管理局（「FDA」）管理的上市前煙草產品申請
（「PMTA」）的要求。董事會注意到FDA於2023年10月12日向R.J. Reynolds Vapor Company
（「RJR」）就其Vuse Alto 品牌下六款薄荷醇口味及混合莓果口味電子煙產品（「產品」）發出了
營銷拒絕令（「MDOs」）。本集團向RJR供應薄荷醇口味產品（「薄荷醇口味產品」），該薄荷醇
口味產品佔本集團2022年1月1日至2023年9月30日期間收入約13%。
董事會進一步注意到RJR同日表示打算對MDOs提出質疑，並將尋求暫緩執行。本公司亦獲
RJR知會，MDOs臨時行政暫緩已經獲得批准，因此薄荷醇口味產品的營銷及分銷在美國可
以繼續。若暫緩執行獲批准，薄荷醇口味產品可以繼續在美國營銷及分銷。若RJR未能獲
得暫緩執行且臨時行政暫緩到期或被撤銷，或者最終MDOs被維持，薄荷醇口味產品可能
不被允許在美國營銷及分銷。本公司將密切關注事態發展並適時作出進一步公告。
–1–本公司股東及投資者在買賣本公司股份時務請謹慎行事，如有疑問，應向專業或財務顧問
尋求獨立意見。
承董事會命
思摩爾國際控股有限公司
陳志平先生
董事會主席
香港，2023年10月15日
於本公告日期，本公司執行董事為陳志平先生，熊少明先生，王貴升先生及王鑫女士；非執行董事為江敏女
士；獨立非執行董事為鍾山先生，閻小穎先生和王高博士。
–2–


In [14]:
def tc_to_sc(text):
    """
    traditional chinese to simplified chinese
    """
    cc = OpenCC('t2s')
    simplified_text = cc.convert(text)
    return simplified_text

In [16]:
llm_config = get_llm_config(trail_id=50)
llm_config["functions"] = [
    {
        "name": "tc_to_sc",
        "description": "convert Traditional Chinese text to Simplified Chinese text.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "Only input parameter for the function. It is a string in Traditional Chinese that you want to convert to Simplified Chinese.",
                }
            },
            "required": ["text"],
        },
    },
    {
        "name": "extract_text_from_pdf",
        "description": "Designed to extract text from a PDF file. Opens the file through a filepath, reads it using a PDF reader, then extract text.",
        "parameters": {
            "type": "object",
            "properties": {
                "file": {
                    "type": "string",
                    "description": "A string representing the path to the PDF file from which the text is to be extracted."
                }
            }
        }
    }
]

assistant_for_jarvix = autogen.AssistantAgent(
    name="assistant_for_jarvix",
    # system_message="You are a helpful assistant. Reply TERMINATE when the task is done.",
    llm_config=llm_config
)

# 我
jarvix = autogen.UserProxyAgent(
    name="jarvix_proxy",
    human_input_mode="NEVER",#"TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "jarvix_proxy",
                           "use_docker": False},
    function_map={"tc_to_sc": tc_to_sc,
                  "extract_text_from_pdf": extract_text_from_pdf},
)
task3_prompt = """
Extract all text from the PDF file located in {pdf_filepath}. If it is in traditional chinese, transfer to simplified chinese string.
"""
task3_template = PromptTemplate.from_template(task3_prompt)
task3 = task3_template.format(pdf_filepath=os.path.join(download_dir, "hk_notice.pdf"))
print(task3)
jarvix.initiate_chat(assistant_for_jarvix, message=task3)


Extract all text from the PDF file located in /Users/jarvixwang/Documents/Project/autogen/download/hk_notice.pdf. If it is in traditional chinese, transfer to simplified chinese string.

jarvix_proxy (to assistant_for_jarvix):


Extract all text from the PDF file located in /Users/jarvixwang/Documents/Project/autogen/download/hk_notice.pdf. If it is in traditional chinese, transfer to simplified chinese string.


--------------------------------------------------------------------------------
assistant_for_jarvix (to jarvix_proxy):

***** Suggested function Call: extract_text_from_pdf *****
Arguments: 
{
  "file": "/Users/jarvixwang/Documents/Project/autogen/download/hk_notice.pdf"
}
**********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION extract_text_from_pdf...
jarvix_proxy (to assistant_for_jarvix):

***** Response from calling function "extract_text_from_pdf" *****
香港交